<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
from IPython.display import IFrame
url = 'http://localhost:3000/colabasaservice/?insidecolab=true'
IFrame(src='http://localhost:3000/colabasaservice/?insidecolab=true', width=700, height=600)

In [3]:
%%javascript

document.onmessage = (...message) => {
  console.log("Cell: Roger. I received this messag:", ...mwssage)
}

const iframe = document.createElement('iframe');
iframe.src = "https://voodoohop.github.io/colabasaservice/";
iframe.width = 700
iframe.height = 600
iframe.frameBorder = 0
document.body.appendChild(iframe);


<IPython.core.display.Javascript object>

In [ ]:

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

In [3]:
!pip install deep-daze --upgrade
!pip install stomp.py imageio-ffmpeg asyncio websockets --upgrade

     |████████████████████████████████| 1.4MB 4.6MB/s 
     |████████████████████████████████| 92kB 7.7MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 3.3MB 12.6MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=b6ec63bb722d0130ee18fca2acbc83b82d29045bfddf60576d4cb4cab4fbc28a
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
  Created wheel for ftfy: filename=ftfy-6.0.1-cp37-none-any.whl size=41573 sha256=fb5ef1080d40d1ca84fb4767a2f0254b3060cbf88467ffa655e434d3bd7738a8
  Stored in directory: /root/.cache/pip/wheels/ae/73/c7/9056e14b04919e5c262fe80b54133b1a88d73683d05d7ac65c
Successfully built fire ftfy
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling image

In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 

flatten = itertools.chain.from_iterable

TEXT="A black and white technical illustration of merging a cat and a human through biohacking."

DEFAULT_PARAMS='--lower_bound_cutout 0.5 --image-width=512 --iterations=5000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123'

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)



def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  cmd = f'imagine {params} "{text}"'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)

    headers = { 
        "params": params,
        "text": TEXT
    }
    conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files

    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          filename = os.path.basename(added)
          mimetype, _ = guess_type(filename)
          print("Adding:", added,"type",mimetype)
          
          with open(added,"rb") as media_file:
            added_media = media_file.read()

            base64_encoded_media = f"data:{mimetype};base64,{b64encode(added_media).decode('ascii')}"
          
          headers = {
              **headers,         
              "type": mimetype,
              "filename": filename
              }
          conn.send(body=base64_encoded_media, headers=headers, destination='/topic/colabMediaOut')

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))
run(TEXT)
conn.disconnect()



epochs:   0%|          | 0/2 [00:00<?, ?it/s]
                                             
Adding: ./A_black_and_white_technical_illustration_of_merging_a_cat_and_a_human_through.jpg type image/jpeg
loss: -21.08:   0%|          | 1/5000 [00:03<2:13:39,  1.60s/it]
                                             
Adding: ./A_black_and_white_technical_illustration_of_merging_a_cat_and_a_human_through.000001.jpg type image/jpeg
loss: -21.05:   0%|          | 3/5000 [00:06<2:11:22,  1.58s/it]
                                             
Adding: ./A_black_and_white_technical_illustration_of_merging_a_cat_and_a_human_through.000002.jpg type image/jpeg
loss: -24.13:   0%|          | 5/5000 [00:09<2:11:14,  1.58s/it]
                                             
Adding: ./A_black_and_white_technical_illustration_of_merging_a_cat_and_a_human_through.000003.jpg type image/jpeg
loss: -26.84:   0%|          | 7/5000 [00:12<2:10:44,  1.57s/it]
                                             
Adding: ./A

# New section

In [ ]:
%%javascript
// Create the channel object
const channel = new BroadcastChannel('my-channel');

// Set up a listener
channel.onmessage = message => console.log('Received:', message.data);

// Post a message to all subscribers
channel.postMessage('Hello, world');

In [ ]:
conn.disconnect()

In [ ]:
!pip install simple-websocket-server

In [ ]:
from simple_websocket_server import WebSocketServer, WebSocket


class SimpleEcho(WebSocket):
    def handle(self):
        # echo message back to client
        self.send_message(self.data)

    def connected(self):
        print(self.address, 'connected')

    def handle_close(self):
        print(self.address, 'closed')


server = WebSocketServer('', 8000, SimpleEcho)
server.serve_forever()
loop = asyncio.get_event_loop()
loop.create_task(some_async_function())